# Étude sur les transits de Flexis

Les résultats détaillés sont disponibles sur cette page :  
https://www.notion.so/bib-batteries/Pilote-Flexis-28b2de3b75c780059a63c20659b769a9


In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
from core.sql_utils import *
from core.s3.s3_utils import S3Service
from core.s3.settings import S3Settings
from core.spark_utils import create_spark_session
from sqlalchemy import text
import pyspark.sql.functions as F
from core.stats_utils import weighted_mean

settings = S3Settings()

spark = create_spark_session(
    settings.S3_KEY,
    settings.S3_SECRET
)
s3 = S3Service()

## Chargement des données

In [ ]:
ford_raw = s3.read_parquet_df_spark(spark, 'raw_ts/ford/time_series/raw_ts_spark.parquet').toPandas()

In [ ]:
flexis_vin = ('WF0AXXTTRBPK69598','WF0AXXTTRBPK69618','WF0AXXTTRBPK69622','WF0AXXTTRBPK69633','WF0AXXTTRBPK69637','WF0AXXTTRBPK69642','WF0AXXTTRBPK69652','WF0AXXTTRBPK69958',
              'WF0AXXTTRBPK70767','WF0AXXTTRBPU07135','WF0AXXTTRBPU07175','WF0AXXTTRBPU07195','WF0AXXTTRBPU07256','WF0AXXTTRBPU07329','WF0AXXTTRBPU07408','WF0AXXTTRBPU07427')

In [ ]:
flexis_raw = ford_raw[ford_raw['vin'].isin(flexis_vin)]
flexis_raw['battery_level'] = flexis_raw['battery_level'].astype(float)

In [ ]:
# Récupérer les phases pour chaque vin
df_raw = pd.DataFrame()

for vin in flexis_raw['vin'].unique():
    print(f"Traitement du VIN: {vin}")
    
    # Données brutes pour ce VIN
    df_temp = flexis_raw[flexis_raw['vin'] == vin].copy()
    
    # Chargement des phases traitées
    phase_vin = s3.read_parquet_df_spark(
        spark, 
        f'processed_phases/processed_phases_ford.parquet/VIN={vin}'
    ).toPandas()
    phase_vin['VIN'] = vin
    temp = df_temp.merge(
        phase_vin, 
        left_on='vin', 
        right_on='VIN', 
        how='left'
    )
    
    # Conversion des dates
    temp["date"] = pd.to_datetime(temp["date"], errors="coerce")
    temp["DATETIME_BEGIN"] = pd.to_datetime(
        temp["DATETIME_BEGIN"], 
        errors="coerce"
    )
    temp["DATETIME_END"] = pd.to_datetime(
        temp["DATETIME_END"], 
        errors="coerce"
    )

    # Filtrage : date comprise entre DATETIME_BEGIN et DATETIME_END
    mask = (
        (temp["date"] >= temp["DATETIME_BEGIN"]) & 
        (temp["date"] <= temp["DATETIME_END"])
    )
    df_filtered = temp[mask].copy()
    df_raw = pd.concat([df_raw, df_filtered])
    

df_raw['charger_voltage'] = df_raw['charger_voltage'].astype(float)

In [ ]:
engine = get_sqlalchemy_engine()
con = engine.connect()

with engine.connect() as connection:
    dbeaver_df = pd.read_sql(
        text(f"""
            SELECT v.vin, vd.timestamp, vd.odometer, vd.soh, vd.cycles, vd.consumption, b.net_capacity, b.capacity
            FROM vehicle_data vd
            JOIN vehicle v ON v.id = vd.vehicle_id
            JOIN vehicle_model vm ON vm.id = v.vehicle_model_id
            JOIN battery b ON b.id = vm.battery_id
            WHERE vm.oem_id = '437e8d1a-0200-41da-a06b-4400c4fa5720'
            AND v.vin IN {flexis_vin};
        """),
        con
    )

## 1. Résultats SoH (State of Health)

In [ ]:
px.scatter(
    dbeaver_df, 
    x='odometer', 
    y='soh', 
    color='vin',
    title='SoH en fonction du kilométrage par VIN',
    labels={'odometer': 'Kilométrage (km)', 'soh': 'SoH'}
)

In [ ]:
stats_vin = (
    dbeaver_df.groupby('vin', as_index=False).agg(
        soh=('soh', 'min'),                    
        soh_first=('soh', 'max'),              
        odometer_last=('odometer', 'max'),           
        odometer_first=('odometer', 'min'),    
        timestamp=('timestamp', 'max'),
        cycles=('cycles', 'max'),
        consumption_avg=('consumption', 'mean'),
        consumption_max=('consumption', 'max'),
        consumption_min=('consumption', 'min')
    )
    .eval('odometer_diff = odometer_last - odometer_first')
    .eval('soh_diff = soh - soh_first')
    .eval('degradation_per_10k = ((1 - soh) / odometer_last) * 10000')
)


### 1.1 Analyse des véhicules

In [ ]:

if len(stats_vin) > 0:
    pire_vehicule = stats_vin.loc[stats_vin['soh'].idxmin()]
    meilleur_vehicule = stats_vin.loc[stats_vin['soh'].idxmax()]
    vieillissement_moyen_per_10k = stats_vin['degradation_per_10k'].mean()
    difference_soh = meilleur_vehicule['soh'] - pire_vehicule['soh']
    
    soh_pire_pct = (
        pire_vehicule['soh'] * 100 
        if pire_vehicule['soh'] < 1 
        else pire_vehicule['soh']
    )
    soh_meilleur_pct = (
        meilleur_vehicule['soh'] * 100 
        if meilleur_vehicule['soh'] < 1 
        else meilleur_vehicule['soh']
    )
    degradation_pire_10k = (
        pire_vehicule['degradation_per_10k'] * 100 
        if abs(pire_vehicule['degradation_per_10k']) < 1 
        else pire_vehicule['degradation_per_10k']
    )
    degradation_meilleur_10k = (
        meilleur_vehicule['degradation_per_10k'] * 100 
        if abs(meilleur_vehicule['degradation_per_10k']) < 1 
        else meilleur_vehicule['degradation_per_10k']
    )
    vieillissement_moyen = (
        vieillissement_moyen_per_10k * 100 
        if abs(vieillissement_moyen_per_10k) < 1 
        else vieillissement_moyen_per_10k
    )
    diff_soh_pct = (
        difference_soh * 100 
        if abs(difference_soh) < 1 
        else difference_soh
    )

    # Affichage des résultats
    print("Pire véhicule:")
    print(f"  VIN: {pire_vehicule['vin']}")
    print(f"  SoH actuel: {soh_pire_pct:.2f}%")
    print(f"  Kilométrage: {pire_vehicule['odometer_last']:.0f} km")
    print(f"  Dégradation par 10k km: {degradation_pire_10k:.2f}%")
    
    print("\nMeilleur véhicule:")
    print(f"  VIN: {meilleur_vehicule['vin']}")
    print(f"  SoH actuel: {soh_meilleur_pct:.2f}%")
    print(f"  Kilométrage: {meilleur_vehicule['odometer_last']:.0f} km")
    print(f"  Dégradation par 10k km: {degradation_meilleur_10k:.2f}%")
    
    print("\nMoyenne:")
    print(f"  Vieillissement moyen par 10 000 km: {vieillissement_moyen:.2f}%")
    print(f"  Différence SoH entre meilleur et pire: {diff_soh_pct:.2f} points")
    print("=" * 50)

    resultats_soh = {
        'pire_vin': pire_vehicule['vin'],
        'pire_soh': soh_pire_pct,
        'pire_km': pire_vehicule['odometer_last'],
        'pire_degradation_10k': degradation_pire_10k,
        'meilleur_vin': meilleur_vehicule['vin'],
        'meilleur_soh': soh_meilleur_pct,
        'meilleur_km': meilleur_vehicule['odometer_last'],
        'meilleur_degradation_10k': degradation_meilleur_10k,
        'vieillissement_moyen': vieillissement_moyen,
        'difference': diff_soh_pct
    }


In [ ]:
# Visualisation de la dégradation par 10k km
fig = px.scatter(
    stats_vin, 
    x='odometer_last', 
    y='degradation_per_10k', 
    color='vin',
    title='Dégradation par 10k km en fonction du kilométrage',
    labels={
        'odometer_last': 'Kilométrage (km)', 
        'degradation_per_10k': 'Dégradation par 10k km (%)'
    }
)
fig.show()

In [ ]:
# Visualisation SoH final par kilométrage
fig = px.scatter(
    stats_vin, 
    x='odometer_last', 
    y='soh', 
    color='vin',
    title='SoH final en fonction du kilométrage',
    labels={'odometer_last': 'Kilométrage (km)', 'soh': 'SoH'}
)
fig.show()

## 2. Exploration de raw

In [ ]:
df_raw = pd.DataFrame()

for vin in flexis_raw['vin'].unique():
    print(f"Traitement du VIN: {vin}")
    
    # Données brutes pour ce VIN
    df_temp = flexis_raw[flexis_raw['vin'] == vin].copy()
    
    # Chargement des phases traitées
    phase_vin = s3.read_parquet_df_spark(
        spark, 
        f'processed_phases/processed_phases_ford.parquet/VIN={vin}'
    ).toPandas()
    phase_vin['VIN'] = vin
    temp = df_temp.merge(
        phase_vin, 
        left_on='vin', 
        right_on='VIN', 
        how='left'
    )
    
    # Conversion des dates
    temp["date"] = pd.to_datetime(temp["date"], errors="coerce")
    temp["DATETIME_BEGIN"] = pd.to_datetime(
        temp["DATETIME_BEGIN"], 
        errors="coerce"
    )
    temp["DATETIME_END"] = pd.to_datetime(
        temp["DATETIME_END"], 
        errors="coerce"
    )

    # Filtrage : date comprise entre DATETIME_BEGIN et DATETIME_END
    mask = (
        (temp["date"] >= temp["DATETIME_BEGIN"]) & 
        (temp["date"] <= temp["DATETIME_END"])
    )
    df_filtered = temp[mask].copy()
    df_raw = pd.concat([df_raw, df_filtered])

### 2.1. Impact de la puissance de charge

In [ ]:
# Calcul de la durée entre les mesures
df_raw['time_duration'] = (
    df_raw.sort_values('date')
    .groupby(['VIN'])
    .apply(lambda g: g['date'].diff().dt.total_seconds())
    .values
)

In [ ]:
# Calcul de la tension moyenne pondérée par phase
df_raw['charger_voltage'] = df_raw['charger_voltage'].astype(float)
result = (
    df_raw.groupby(['VIN', 'PHASE_INDEX'])
    .apply(
        lambda g: weighted_mean(
            g['charger_voltage'], 
            g['time_duration']
        )
    )
    .reset_index(name='weighted_charger_voltage')
)

In [ ]:
# Fusion des résultats avec les données brutes
df_raw = result.merge(
    df_raw, 
    left_on=['VIN', 'PHASE_INDEX'], 
    right_on=['VIN', 'PHASE_INDEX'], 
    how='right'
)

In [ ]:
# Séparation des phases de charge et de décharge
df_charging = df_raw[df_raw['PHASE_STATUS'] == 'charging'].copy()
df_discharging = df_raw[df_raw['PHASE_STATUS'] == 'discharging'].copy()

In [ ]:
# Analyse des charges à haute intensité par VIN

# Définition du seuil pour charge à haute intensité
# Pour les véhicules électriques, une charge rapide utilise généralement ≥ 400V
seuil_haute_intensite = 300  # Volts

# Calculer la tension moyenne par phase pour chaque VIN
voltage_par_phase = df_charging.groupby(['VIN', 'PHASE_INDEX']).agg(
    weighted_charger_voltage_mean=('weighted_charger_voltage', 'mean')
).reset_index()

# Identifier les phases à haute intensité
voltage_par_phase['is_haute_intensite'] = (
    voltage_par_phase['weighted_charger_voltage_mean'] >= seuil_haute_intensite
)

# Compter les charges totales et les charges à haute intensité par VIN
stats_charges = voltage_par_phase.groupby('VIN').agg(
    total_charges=('PHASE_INDEX', 'nunique'),      # Nombre unique de phases de charge
    charges_haute_intensite=('is_haute_intensite', 'sum')  # Nombre de phases à haute intensité
).reset_index()

# Calculer la proportion
stats_charges['proportion_haute_intensite'] = (
    stats_charges['charges_haute_intensite'] / 
    stats_charges['total_charges'] * 100
)

# Ajouter des statistiques supplémentaires sur la tension moyenne par VIN
stats_voltage = df_charging.groupby('VIN').agg(
    voltage_moyen=('weighted_charger_voltage', 'mean'),
    voltage_median=('weighted_charger_voltage', 'median'),
    voltage_max=('weighted_charger_voltage', 'max'),
    voltage_min=('weighted_charger_voltage', 'min')
).reset_index()

stats_charges = stats_charges.merge(stats_voltage, on='VIN', how='left')

# Tri par proportion décroissante
stats_charges = stats_charges.sort_values(
    'proportion_haute_intensite', 
    ascending=False
)

# Affichage des statistiques
display(
    stats_charges[[
        'VIN', 'total_charges', 'charges_haute_intensite', 
        'proportion_haute_intensite', 'voltage_moyen', 'voltage_median', 
        'voltage_max', 'voltage_min'
    ]]
)

# Fusion avec les statistiques SoH
stats_charges = stats_charges.merge(
    stats_vin, 
    left_on='VIN', 
    right_on='vin', 
    how='left'
)

# Visualisation
fig = px.bar(
    stats_charges, 
    x='VIN', 
    y='proportion_haute_intensite',
    title=f'Proportion de charges à haute intensité (≥ {seuil_haute_intensite}V) par VIN',
    labels={
        'proportion_haute_intensite': 'Proportion (%)', 
        'VIN': 'VIN'
    },
    color='soh',
    hover_data=['soh', 'odometer']
)
fig.update_layout(showlegend=False, height=500)
fig.show()


In [ ]:
# Résumé des statistiques de charge haute intensité
print("=" * 60)
print("Résumé des charges à haute intensité:")
print(f"  Proportion moyenne: {stats_charges['proportion_haute_intensite'].mean():.2f}%")
print(f"  VIN avec le plus de charges haute intensité: {stats_charges.iloc[0]['VIN']} ({stats_charges.iloc[0]['proportion_haute_intensite']:.2f}%)")
print(f"  VIN avec le moins de charges haute intensité: {stats_charges.iloc[-1]['VIN']} ({stats_charges.iloc[-1]['proportion_haute_intensite']:.2f}%)")
print("=" * 60)

### 2.2. Impact du SoH sur les distances parcourues

In [ ]:
# Chargement des phases traitées
processed_phases = s3.read_parquet_df_spark(
    spark, 
    'processed_phases/processed_phases_ford.parquet'
).toPandas()

# Filtrage pour les VIN Flexis et les phases de décharge
processed_phases_flexis = processed_phases[
    processed_phases['VIN'].isin(flexis_vin)
]
processed_phases_discharging = processed_phases_flexis[
    processed_phases_flexis['PHASE_STATUS'] == 'discharging'
].copy()

# Conversion des colonnes en float
processed_phases_discharging[
    ['ODOMETER_FIRST', 'ODOMETER_LAST', 'SOC_FIRST', 'SOC_LAST']
] = processed_phases_discharging[
    ['ODOMETER_FIRST', 'ODOMETER_LAST', 'SOC_FIRST', 'SOC_LAST']
].astype(float)

# Calcul des différences
processed_phases_discharging['ODOMETER_DIFF'] = (
    processed_phases_discharging['ODOMETER_LAST'] - 
    processed_phases_discharging['ODOMETER_FIRST']
)
processed_phases_discharging['KM/SOC'] = (
    processed_phases_discharging['ODOMETER_DIFF'] / 
    processed_phases_discharging['SOC_DIFF'].abs()
)


In [ ]:
# Filtrage des décharges : entre 5 et 200 km pour éviter les outliers
processed_phases_discharging = processed_phases_discharging[
    (processed_phases_discharging['ODOMETER_DIFF'] > 5) & 
    (processed_phases_discharging['ODOMETER_DIFF'] < 200)
].copy()

In [ ]:
# Calcul des statistiques de décharge par VIN
stats_decharge_vin = processed_phases_discharging.groupby('VIN').agg(
    nb_decharges=('PHASE_INDEX', 'nunique'),
    km_parcourus_moyen=('ODOMETER_DIFF', 'mean'),
    km_parcourus_median=('ODOMETER_DIFF', 'median'),
    soc_perdu_moyen=('SOC_DIFF', 'mean'),
    soc_perdu_total=('SOC_DIFF', 'sum'),
    ratio_km_soc_moyen=('KM/SOC', 'mean'),  
    ratio_km_soc_median=('KM/SOC', 'median'),
    odometer=('ODOMETER_LAST', 'max'),
).reset_index()

In [ ]:
# Fusion avec les statistiques SoH
stats = stats_decharge_vin.merge(
    stats_vin, 
    left_on='VIN', 
    right_on='vin', 
    how='left'
)

In [ ]:
stats_vin

In [ ]:
stats

In [ ]:
# Résumé des statistiques de décharge
print("=" * 60)
print("Statistiques de décharge:")
print(f"  Kilométrage moyen parcouru par décharge: {stats['km_parcourus_moyen'].mean():.2f} km")
print(f"  SoC moyen perdu par décharge: {stats['soc_perdu_moyen'].mean():.2f} points")
print(f"  Ratio moyen km/SOC perdu (tous VIN): {stats['ratio_km_soc_moyen'].mean():.2f} km/point de SoC")
print("=" * 60)

# Visualisation du ratio moyen par VIN
fig1 = px.bar(
    stats.sort_values('ratio_km_soc_median', ascending=False),
    x='VIN',
    y='ratio_km_soc_median',
    title='Ratio moyen km/SOC perdu par VIN',
    labels={
        'ratio_km_soc_median': 'Ratio (km/point de SoC)', 
        'VIN': 'VIN'
    },
    color='soh',
    color_continuous_scale='RdYlGn',
    hover_data=['soh', 'odometer']
)
fig1.update_layout(
    height=500, 
    xaxis_tickangle=-45, 
    showlegend=False
)
fig1.show()
